# Lego Tile User Workflow

This notebook demonstrates the complete workflow for interacting with the Lego tile services. It shows how to:
1. Authenticate using OIDC
2. Claim a tile to build
3. Release a tile (if unable to complete)
4. Commit a tile (when completed)

**Note**: For web developers implementing this workflow in React, see the [auth_example.jsx](auth_example.jsx) file for a complete authentication implementation including PKCE flow and token renewal.

## Important Notes

- **Tile Ownership**: Each tile can only be claimed by one user at a time
- **User Verification**: Only the user who claimed a tile can release or commit it
- **State Management**: Tiles progress through states: available → claimed → released/submitted
- **Grid Dimensions**: 26x27 tiles at zoom level 7

## Setup
First, we'll import the required libraries and set up our connection.

In [10]:
import openeo
import requests

# Backend URL - adjust this to your deployment
base_url = "https://openeo.ds.io"


# Connect to the back-end
connection = openeo.connect(base_url)

## Authentication

Authentication is required for claiming, releasing, and committing tiles. The system uses OpenID Connect (OIDC) with PKCE (Proof Key for Code Exchange) for secure authentication.

The system uses the Copernicus Data Space Ecosystem (CDSE) identity provider:
```
https://identity.dataspace.copernicus.eu/auth/realms/CDSE/.well-known/openid-configuration
```

When you run the authentication cell, it will open a browser window for you to log in with your OIDC provider.

In [11]:
# Authenticate using OIDC
connection.authenticate_oidc_authorization_code(client_id="cdse-public")

# Store the token for later use
token = connection.auth.bearer

## Claiming a Tile

When claiming a tile, the service will:
1. Assign you an available tile from the grid
2. Generate building instructions for that tile
3. Lock the tile to prevent others from claiming it

The response will be a PNG image containing the building instructions.

**Note**: If you already have a claimed tile, the service will return that tile's instructions instead of assigning a new one.

In [4]:
claim_service_id = "9c1af4a8-7a9f-4c83-a6c8-2809e347075c"
zoom_level = 7

# Call the claim service
# Note: x=0, y=0 are placeholder values - the service will assign an available tile
response = requests.get(
    f"{base_url}/services/xyz/{claim_service_id}/tiles/{zoom_level}/0/0",
    headers={"Authorization": f"Bearer {token}"},
)

# Display the tile instructions
if response.status_code == 200:
    from PIL import Image
    from IPython.display import display
    from io import BytesIO

    img = Image.open(BytesIO(response.content))
    display(img)
else:
    print("Error:", response.status_code)
    print(response.text)
    
    # Common error cases:
    # 401 - Not authenticated
    # 403 - Not authorized
    # 409 - No tiles available

Error: 500
{"code":"ServerError","message":"module 'colour' has no attribute 'Color'"}


## Same in JSON

The following code snippet shows how to claim a tile and get the building instructions in JSON format. The response will include the tile ID, coordinates, and the building instructions in JSON format.

In [ ]:
claim_service_json_id = "ce714e87-0df0-4d74-b334-ff56be9f48bb"
zoom_level = 7

# Call the claim service
# Note: x=0, y=0 are placeholder values - the service will assign an available tile
response = requests.get(
    f"{base_url}/services/xyz/{claim_service_json_id}/tiles/{zoom_level}/0/0",
    headers={"Authorization": f"Bearer {token}"},
)

# Display the tile instructions
if response.status_code == 200:
    # print pretty JSON
    import json
    print(json.dumps(response.json(), indent=2))
else:
    print("Error:", response.status_code)
    print(response.text)
    
    # Common error cases:
    # 401 - Not authenticated
    # 403 - Not authorized
    # 409 - No tiles available

SyntaxError: invalid syntax (4155700469.py, line 15)

## Releasing a Tile

If you are unable to complete your assigned tile, you can release it back to the pool for others to claim.
This should be done if you:
- Don't have the required brick colors
- Don't have time to complete the build
- Encounter any issues preventing completion

**Important**: 
- You can only release tiles that you have claimed
- You cannot release a tile that has already been submitted
- Released tiles become available for other users to claim

In [9]:
release_service_id = "bdd95301-d21c-4133-80a0-6f2321e9116d"

# Call the release service
# Note: x=0, y=0 are placeholder values - the service will release your assigned tile
response = requests.get(
    f"{base_url}/services/xyz/{release_service_id}/tiles/{zoom_level}/0/0",
    headers={"Authorization": f"Bearer {token}"},
)

if response.status_code == 200:
    print("Tile successfully released")
    print(response.text)
else:
    print("Error:", response.status_code)
    print(response.text)
    
    # Common error cases:
    # 401 - Not authenticated
    # 403 - Not authorized
    # 404 - No tile assigned to release
    # 409 - Cannot release (e.g., tile already submitted)

Error: 409
{"code":"TileAlreadyLocked","message":"Tile 72/49/7 is already locked for user 9d062e90-5cbc-4024-a130-778a0241105e for service lego-mosaic-test-1"}


## Committing a Tile

Once you have completed building your tile, you should commit it to mark it as finished.
This will:
1. Permanently mark the tile as completed
2. Prevent the tile from being released or reassigned
3. Update the overall map progress

**Important Requirements**:
- You must be the user who claimed the tile
- The tile must be in a 'claimed' state (not released or already submitted)
- You should only commit after:
  - Fully completing the build according to instructions
  - Verifying all colors and placements
  - Attaching the tile to the main map in the correct position

**Warning**: Committing a tile is permanent and cannot be undone!

In [10]:
commit_service_id = "ba11ca30-8b26-4076-afa4-b67736b0f0aa"

# Call the commit service
# Note: x=0, y=0 are placeholder values - the service will commit your assigned tile
response = requests.get(
    f"{base_url}/services/xyz/{commit_service_id}/tiles/{zoom_level}/0/0",
    headers={"Authorization": f"Bearer {token}"},
)

if response.status_code == 200:
    print("Tile successfully committed")
    print(response.text)
else:
    print("Error:", response.status_code)
    print(response.text)
    
    # Common error cases:
    # 401 - Not authenticated
    # 403 - Not authorized
    # 404 - No tile assigned to commit
    # 409 - Cannot commit (e.g., tile already submitted or released)

Error: 409
{"code":"TileAlreadyLocked","message":"Tile 72/49/7 is already locked for user 9d062e90-5cbc-4024-a130-778a0241105e for service lego-mosaic-test-1"}


## Error Handling

The services use standard HTTP status codes to indicate success or failure:

- **200**: Success
- **401**: Not authenticated (missing or invalid token)
- **403**: Not authorized (insufficient permissions)
- **404**: Resource not found (e.g., no tile assigned)
- **409**: Conflict (e.g., no tiles available, tile already claimed)

When an error occurs, the response will include a detailed message explaining the issue.